# Build MCP from OpenAPI Specification with Bedrock AgentCore Gateway

This workshop demonstrates how to use Amazon Bedrock AgentCore Gateway to automatically generate Model Context Protocol (MCP) servers from OpenAPI specifications, enabling seamless integration of external APIs with AI agents.

## Overview

In this lab, you will:
- Create a Bedrock AgentCore Gateway with authentication
- Configure OpenAPI-based gateway targets using external API specifications
- Set up secure credential management for external API access
- Deploy and test the gateway with Strands Agents
- Explore automatic MCP tool generation from OpenAPI schemas

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of Strands Agents and Bedrock AgentCore concepts
- An external API key (e.g., Exa API key) for testing

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents and Bedrock AgentCore Python SDK:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore rich

## What is Bedrock AgentCore Gateway?

Amazon Bedrock AgentCore Gateway is a managed service that automatically converts OpenAPI specifications into Model Context Protocol (MCP) servers. Key benefits include:

- **Automatic Tool Generation**: Converts OpenAPI endpoints into MCP tools without manual coding
- **Secure Authentication**: Built-in support for various authentication methods (API keys, JWT, OAuth)
- **Managed Infrastructure**: No need to deploy or maintain custom MCP servers
- **Scalability**: Automatically scales based on demand
- **Integration**: Seamless integration with existing REST APIs

This approach allows you to quickly integrate any REST API with OpenAPI documentation into your AI agents as tools.

![openapi-gateway-apikey.png](images/openapi-gateway-apikey.png)

## Creating AgentCore Gateway with OpenAPI as Target

### Step 1: Inbound Authentication - Setting up Amazon Cognito for Gateway Authentication

Create a Cognito User Pool for secure access to the AgentCore Gateway. This provides JWT-based authentication for the gateway endpoints.

**Components Created:**
- **User Pool**: Manages user identities and authentication
- **App Client**: Enables application-level authentication
- **Test User**: For testing the authentication flow

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Initialize Cognito client
cognito_client = boto3.client('cognito-idp', region_name=region)

# Create User Pool
user_pool_response = cognito_client.create_user_pool(
    PoolName='MCPServerPool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 8
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# Create App Client
app_client_response = cognito_client.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='MCPServerPoolClient',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# Create User
cognito_client.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# Set Permanent Password
cognito_client.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password='MyPassword123!',
    Permanent=True
)

# Output the required values
print(f"Pool id: {cognito_pool_id}")
print(f"Discovery URL: https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration")
print(f"Client ID: {cognito_client_id}")

### Step 2: Creating IAM Role for AgentCore Gateway

Create an IAM role that the AgentCore Gateway will assume to access AWS services and manage the gateway operations.

In [ ]:
import boto3
import json

# Create IAM client
iam_client = boto3.client('iam')

# Define the trust policy for the role
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role
role_name = "BedrockAgentCoreGatewayRole"
description = "IAM role for Bedrock Agent with BedrockAgentCoreFullAccess policy"

try:
    iam_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description=description
    )
    
    print(f"Role created successfully: {iam_role['Role']['Arn']}")
    
    # Attach the BedrockAgentCoreFullAccess managed policy
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/BedrockAgentCoreFullAccess"
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess"
    )
    
    print(f"Policy attached successfully to role: {role_name}")
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"Role {role_name} already exists")
    iam_role = iam_client.get_role(
        RoleName=role_name
    )
except Exception as e:
    print(f"Error creating role: {str(e)}")


### Step 3: Creating the AgentCore Gateway

Create the main AgentCore Gateway with Cognito JWT authentication. This gateway will host our OpenAPI-based targets.

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
from botocore.exceptions import ClientError
import boto3

region = boto3.session.Session().region_name

gateway_client = boto3.client('bedrock-agentcore-control', region_name=region)
gateway_name = 'SampleAgentCoreGateway'

auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [cognito_client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration"
    }
}
try:
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
        protocolType='MCP',
        authorizerType='CUSTOM_JWT',
        authorizerConfiguration=auth_config, 
        description='AgentCore Gateway with OpenAPI target'
    )
    # Retrieve the GatewayID used for GatewayTarget creation
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    print(f"Gateway ID: {gateway_id}")
    print(f"Gateway URL: {gateway_url}")
except ClientError as e:
    print(f"❌ ERROR: {e}")
    if "already exists" in str(e):
        # If gateway already exists, retrieve its ID
        for item in gateway_client.list_gateways()['items']:
            if item['name'] == gateway_name:
                gateway = gateway_client.get_gateway(gatewayIdentifier=item['gatewayId'])
                gateway_id = gateway['gatewayId']
                gateway_url = gateway['gatewayUrl']
                print(f"Gateway ID: {gateway_id}")
                print(f"Gateway URL: {gateway_url}")
                break
except Exception as e:
    # Show any errors during memory creation
    print(f"❌ ERROR: {e}")


### Step 4: Preparing the OpenAPI Specification

Download the Exa API OpenAPI specification that will be used to automatically generate MCP tools. The OpenAPI spec defines all available endpoints, parameters, and response schemas.

In [ ]:
import requests
import os

url = "https://raw.githubusercontent.com/exa-labs/openapi-spec/refs/heads/master/exa-openapi-spec.yaml"
openapi_file_name = url.split("/")[-1]
save_path = f"./{openapi_file_name}"

if os.path.exists(save_path):
    print(f"File already exists: {save_path}")
else:
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded successfully: {save_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

### Step 5: Uploading OpenAPI Spec to S3

Upload the OpenAPI specification to S3 so that the AgentCore Gateway can access it for automatic tool generation.

In [ ]:
import boto3

region = boto3.session.Session().region_name
# Create an S3 client
s3_client = boto3.client('s3', region_name=region)
sts_client = boto3.client('sts', region_name=region)

account_id = sts_client.get_caller_identity()["Account"]

# Define parameters
# Your s3 bucket to upload the OpenAPI json file.
bucket_name = f'agentcore-gateway-{account_id}-{region}'
file_path = f'./{openapi_file_name}'
object_key = openapi_file_name

# Upload the file using put_object and read response
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # Construct the ARN of the uploaded object with account ID and region
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

### Step 6: Outbound Authentication - Setting up API Key Credential Provider

Create a secure credential provider for the Exa API key using Bedrock AgentCore Identity. This ensures the API key is stored securely and can be retrieved by the gateway when making API calls.

To get Exa API key, go to [Exa login page](https://dashboard.exa.ai/login) to register with your email. 

Then go to [API key section](https://dashboard.exa.ai/api-keys) in Exa dashboard to create an API key. Copy the API key to `EXA_API_KEY` in below code...

**Important**: Replace the placeholder API key with your actual Exa API key.

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
from botocore.exceptions import ClientError
import boto3

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = <YOUR EXA API KEY> 

region = boto3.session.Session().region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "exa-apikey-provider",
        "apiKey": EXA_API_KEY # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
    })
    print("✅ Created AgentCore Identity API Key Credential Provider.")
    print(f"API Key Provider ARN: {api_key_provider['credentialProviderArn']}")
except ClientError as e:
    print(f"❌ ERROR: {e}")
    if "already exists" in str(e):
        # If api key provider already exists, retrieve its ID
        from bedrock_agentcore._utils import endpoints
        
        cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=endpoints.get_control_plane_endpoint(region))
        api_key_provider = cp_client.get_api_key_credential_provider(name="exa-apikey-provider")
        print(f"API Key provider already exists.")
        print(f"API Key Provider ARN: {api_key_provider['credentialProviderArn']}")
except Exception as e:
    # Show any errors during memory creation
    print(f"❌ ERROR: {e}")

### Step 7: Creating Gateway Target with OpenAPI Configuration

Create a gateway target that uses the OpenAPI specification to automatically generate MCP tools. The target configuration includes:

- **OpenAPI Schema**: Reference to the S3-stored OpenAPI specification
- **Credential Configuration**: How to authenticate with the external API
- **Parameter Mapping**: Where to place the API key (query parameter, header, etc.)

In [29]:
from botocore.exceptions import ClientError

gateway_target_name = 'ExaOpenAPITarget'

# S3 Uri for OpenAPI spec file
exa_openapi_s3_target_config = {
    "mcp": {
        "openApiSchema": {
            "s3": { "uri": openapi_s3_uri }
        }
    }
}

# API Key credentials provider configuration
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "x-api-key", # Replace this with the name of the api key name expected by the respective API provider. For passing token in the header, use "Authorization"
                    "providerArn": api_key_provider['credentialProviderArn'],
                    "credentialLocation":"HEADER", # Location of api key. Possible values are "HEADER" and "QUERY_PARAMETER".
                    #"credentialPrefix": " " # Prefix for the token. Valid values are "Basic". Applies only for tokens.
            }
        }
    }
]

try:
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=gateway_target_name,
        description='OpenAPI Target with S3Uri using SDK',
        targetConfiguration=exa_openapi_s3_target_config,
        credentialProviderConfigurations=api_key_credential_config
    )
    gateway_target_id = response['targetId']
    print(f"Gateway Target ID: {gateway_target_id}")
except ClientError as e:
    print(f"❌ ERROR: {e}")
    if "already exists" in str(e):
        # If gateway already exists, retrieve its ID
        for item in gateway_client.list_gateway_targets(gatewayIdentifier=gateway_id)['items']:
            if item['name'] == gateway_target_name:
                gateway_target_id = item['targetId']
                print(f"Gateway Target ID: {gateway_target_id}")
                break
except Exception as e:
    # Show any errors during memory creation
    print(f"❌ ERROR: {e}")

## Testing the Deployed AgentCore Gateway as MCP Server with Strands Agent

Now let's test our deployed AgentCore Gateway as MCP Server with proper authentication.


### Obtaining Access Token from Cognito Authentication

Get the JWT access token from Cognito that will be used to authenticate requests to the AgentCore Gateway.

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Get bearer token (access token) from Cognito Auth 
cognito_client = boto3.client('cognito-idp', region_name=boto3.session.Session().region_name)
auth_response = cognito_client.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']
print(bearer_token)

### Testing the Gateway with Strands Agent

Now let's test our AgentCore Gateway by connecting to it with a Strands Agent. The gateway automatically converts the OpenAPI specification into MCP tools that the agent can use.

**What happens here:**
1. Connect to the gateway using the JWT bearer token
2. List available tools (automatically generated from OpenAPI spec)
3. Create an agent with access to these tools
4. Test the agent's ability to use the external API through the gateway

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Connect to the Web Search MCP server
print("\nConnecting to MCP Server...")
headers = {
    "Authorization": f"Bearer {bearer_token}",
    #"Content-Type": "application/json"
}
exa_server = MCPClient(lambda: streamablehttp_client(gateway_url, headers))

with exa_server:
    mcp_tools = (exa_server.list_tools_sync())
    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with self-built MCP tools
    agent = Agent(
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
        system_prompt = """You are a helpful assistant that provides concise responses.""",
        tools=mcp_tools,
    )

    agent("What is Bedrock AgentCore?")

Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("Agent Loop Detail")
console.rule()
console.print(f"Number of Loops: {agent.event_loop_metrics.cycle_count}")

table = Table(title="Agent Messages", show_lines=True)
table.add_column("Role", style="green")
table.add_column("Text", style="magenta")
table.add_column("Tool Name", style="cyan")
table.add_column("Tool Input", style="cyan")
table.add_column("Tool Result", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## Resource Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
import boto3
import os

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)
iam_client = boto3.client('iam')
s3_client = boto3.client('s3', region_name=region)

try:
    print("Deleting AgentCore Gateway Target...")
    agentcore_control_client.delete_gateway_target(gatewayIdentifier=gateway_id, targetId=gateway_target_id)
    print("✓ AgentCore Gateway Target deleted")
    
    print("Deleting AgentCore Gateway...")
    agentcore_control_client.delete_gateway(gatewayIdentifier=gateway_id)
    print("✓ AgentCore Gateway deletion initiated")

    print("Deleting AgentCore Identity...")
    agentcore_control_client.delete_api_key_credential_provider(name="exa-apikey-provider")
    print("✓ AgentCore Identity deletion initiated")

    print("Deleting Cognito User Pool...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool deleted")

    print("Deleting IAM Role...")
    iam_client.detach_role_policy(RoleName=role_name, PolicyArn="arn:aws:iam::aws:policy/BedrockAgentCoreFullAccess")
    iam_client.detach_role_policy(RoleName=role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess")
    iam_client.delete_role(RoleName=role_name)
    print("✓ IAM Role deleted")

    print("Deleting S3 Bucket...")
    s3_client.delete_object(Bucket=bucket_name, Key=openapi_file_name)
    s3_client.delete_bucket(Bucket=bucket_name)
    print("✓ S3 Bucket deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

## Conclusion

In this lab, you successfully:

- ✅ Created a Bedrock AgentCore Gateway with JWT authentication
- ✅ Configured secure credential management for external APIs
- ✅ Automatically generated MCP tools from OpenAPI specifications
- ✅ Integrated the gateway with Strands Agents for AI-powered API interactions

## Key Benefits of AgentCore Gateway

- **Zero Code MCP Generation**: Automatically converts any OpenAPI spec into MCP tools
- **Secure Credential Management**: Built-in support for various authentication methods
- **Managed Infrastructure**: No need to deploy or maintain custom servers
- **Scalable**: Automatically handles load and scaling
- **Standards-Based**: Works with any REST API that has OpenAPI documentationlonger needed